In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the SimpleCNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1),
            nn.GroupNorm(64, 64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1),
            nn.GroupNorm(64, 64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 4, 2, 1)
        )
        self.classifier = nn.Sequential(
            nn.GroupNorm(64, 64),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Load and preprocess the MNIST dataset
def load_data():
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
    return train_loader, test_loader

# Training function
def train(model, device, train_loader, optimizer, epoch, target_multiplier, epsilon):
    model.train()
    correct = 0
    total = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        # Create progressive target
        progressive_target = torch.zeros(target.size(0), 10, device=device)
        progressive_target.scatter_(1, target.unsqueeze(1), target_multiplier)

        loss = nn.functional.mse_loss(output, progressive_target)

        # Check for equilibrium
        if loss.item() > epsilon:
            loss.backward()
            optimizer.step()

        # Calculate accuracy
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)

    train_accuracy = 100. * correct / total
    print(f'Train Epoch: {epoch} Accuracy: {correct}/{total} ({train_accuracy:.2f}%)')

# Test function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += nn.functional.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    return accuracy

# Main training loop
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SimpleCNN().to(device)
    optimizer = optim.Adam(model.parameters())
    train_loader, test_loader = load_data()

    epsilon = 1e-4  # Equilibrium threshold
    target_multipliers = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
    epochs_per_multiplier = 5

    for t in target_multipliers:
        print(f"\nTraining with target multiplier: {t}")
        for epoch in range(1, epochs_per_multiplier + 1):
            train(model, device, train_loader, optimizer, epoch, t, epsilon)
        accuracy = test(model, device, test_loader)

    print("Training completed.")

if __name__ == '__main__':
    main()



Training with target multiplier: 0.1
Train Epoch: 1 Accuracy: 54059/60000 (90.10%)
Train Epoch: 2 Accuracy: 58928/60000 (98.21%)
Train Epoch: 3 Accuracy: 59217/60000 (98.69%)
Train Epoch: 4 Accuracy: 59353/60000 (98.92%)
Train Epoch: 5 Accuracy: 59412/60000 (99.02%)

Test set: Average loss: 2.2214, Accuracy: 9910/10000 (99.10%)


Training with target multiplier: 0.2
Train Epoch: 1 Accuracy: 59238/60000 (98.73%)
Train Epoch: 2 Accuracy: 59433/60000 (99.06%)
Train Epoch: 3 Accuracy: 59551/60000 (99.25%)
Train Epoch: 4 Accuracy: 59589/60000 (99.31%)
Train Epoch: 5 Accuracy: 59647/60000 (99.41%)

Test set: Average loss: 2.1362, Accuracy: 9936/10000 (99.36%)


Training with target multiplier: 0.3
Train Epoch: 1 Accuracy: 59564/60000 (99.27%)
Train Epoch: 2 Accuracy: 59674/60000 (99.46%)
Train Epoch: 3 Accuracy: 59692/60000 (99.49%)
Train Epoch: 4 Accuracy: 59719/60000 (99.53%)
Train Epoch: 5 Accuracy: 59747/60000 (99.58%)

Test set: Average loss: 2.0532, Accuracy: 9934/10000 (99.34%)


Tra